<h1>Data Visualization</h1>

<p style="font-size: 16px">In this notebook I'm assuming that you have already followed the previous one and signed up to Google Earth Engine and Authenticated your account. So the goal on this notebook is to dive deeper into:</p>
<ul>
    <li>Google Earth Engine structure</li>
    <li>Google's Data Catalog</li>
    <li>Remote Sensing data acquisition</li>
    <li>Image visualization with folium and geehydro</li>
    <li>Filter data</li>    
</ul>

In [1]:
import ee
import folium
import geehydro
from IPython.display import Image

In [2]:
ee.Initialize()

<p style="font-size: 16px">First we have to initialize our Folium map. Here we are setting two basic parameters: location, that is where the center of the map will be located when the map is initialized; zoom_start, that is the zoom level that the map will have when initialize. If you press shift+tab inside the constructor of the function you will see many other parameters that have very intuitive names that you can play with.
After that, we use the setOptions function, that is from geehydro library. geehydro is a package built that implements several methods for creating an interaction between folium and Earth Engine, with it we can add layers to the map and set some controls to it. With setOptions function we are setting the base layer of our map, and we have the following options:</p>
<ul>
    <li>"ROADMAP"
        <ul>
            <li>It's a Google Maps-like map, with road information (obviously)</li>
            <li>It also has information about countries and states' boundaries</li>
        </ul>
    </li>
    <li>"SATELLITE"
        <ul>
            <li>It has satellite image, with no information about countries, boundaries, etc. It's only images</li>
            <li>Images can vary on quality from place to place</li>
        </ul>
    </li>
    <li>"HYBRID"
        <ul>
            <li>It's a mix of SATELLITE and ROADMAP</li>
            <li>You have satellite images with boundaries, roads, states, etc.</li>
        </ul>
    </li>
    <li>"TERRAIN"
        <ul>
            <li>Similar to ROADMAP with soil details</li>
            <li>You can visualize mountains and types of soil</li>
        </ul>
    </li> 
    <li>"NAIP"
        <ul>
            <li>Looks like it's in development yet</li>
            <li>NAIP stands for National Agriculture Imagery Program from US</li>
        </ul>
    </li>
    <li>"ESRI"
        <ul>
            <li>Diffenrent satellite visualization</li>
            <li>Higher resolution images than "SATELLITE"</li>
            <li>In some areas, if you try to zoom in, it might not have the data available</li>
        </ul>
    </li>
</ul>

<p style="font-size: 16px">We can also personalize our map in other ways that we will see further in this series of notebooks. By now, this is enough.</p>

<img src='Assets/maps.png' width='550' height='550' style='margin-left: 550px;'>

In [3]:
# Initializing the folium map
_map = folium.Map(location=[-0.784176, -60.084085], zoom_start=6)
# Setting the type of map we want to have on background
_map.setOptions('ROADMAP')

<h2>Data Catalog</h2>

<p>The Earth Engine API has its own data catalog and Google is doing a great job on providing it in an easy to use and free way. You can find all the data available for Earth Engine in this <a href="https://developers.google.com/earth-engine/datasets">link</a>. In this series we will be working mainly with raw data, e.g. bands of satellite imagery, as our goal is to apply some Machine Learning and Deep Learning techniques using Remote Sensing data.

In the EE's data catalog we can find imagery from Landsat series (1, 4, 5, 7 and 8), sentinel series (1, 2, 3 and 5P), MODIS (too many to put here), Planet labs SkySat and NAIP. Those are very precious data that we will be exploring in details set-by-set. It's very easy to acquire the data using EE, you just need to access the data catalog, search for the dataset you want to use, and in the page of this dataset there will be an Id similar to this: LANDSAT/LC08/C01/T1. Then, you just need to paste it into the ee.ImageCollection() function, that we will be seing in the folling cells.
    
Now we will be taking a closer look at Landsat imagery, more specificly at Landsat-8 imagery. In the <a href="https://developers.google.com/earth-engine/datasets/catalog/landsat-8">Landsat-8 section</a> we find 3 options: Surface Reflectance, Top of Atmosphere and Raw Images. We will understand better each of these sections further, for now let's just say that there are raw images with a lot of noise which need some preprocessing, and top of atmosphere and surface reflectance are images after some preprocessing methods are applied. In this notebook, we will visualize the raw images that have the ID LANDSAT/LC08/C01/T1. This ID is from a ImageCollection, a class that its object contains many images and methods to filter and process those images.
    
In this catalog there are some Feature Collections also. We can find a lot of them with different purposes and we will be using some of them in this series. As an example we gonna use the <a href="https://developers.google.com/earth-engine/datasets/catalog/USDOS_LSIB_SIMPLE_2017">Large Scale International Boundary Polygons, Simplified (LSIB)</a> FeatureCollection to filter images that cover only a country's boundaries. In the data catalog you can find the table schema to understand better how to use the FeatureCollection. Similar to ImageCollection, FeatureCollection is a class that its object has many features, and those features can be countries boundaries, states boundaries, U.S terrestrial and marine protected areas boundaries, global land ice measurements, and many more.
    
So what we gonna do in the next cells is load images from Landsat-8 that cover Brazil's territory. Those images will be a 1-year composite, i.e. we select an image collection from a 1-year time range and select the median value for each pixel, generating only one image. Now let's go to the code
</p>



<img src='Assets/ee_data_catalog.png' width='600' height='500' style='margin-left: 550px; border: 1px solid;'>

<h2>Let's Code</h2>

<p>First, we have to select the FeatureCollection that contains boundaries from all countries. As said before, the ID of this feature collection can be found on the Earth Engine Data Catalog.
    
So, we use the ee.FeatureCollection constructor and the ID to get those boundaries</p>

In [4]:
# Getting the Large Scale International Boundary Polygons FeatureCollection, i.e. getting the boundaries of all countries in the world 
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')

However, we just want to get Brazil's boundaries and the above cell returned to us boundaries from the whole world. Now what we have to do is use a filter to get what we want. An object of the FeatureCollection class has a method called filter, which receives a filter from the Filter class, and this filter filters based on the metadata of the FeatureCollection object. So we are using the ee.Filter.eq() function that filters to metadata equal to the given value (eq stands for equal). 'country_na' and 'Brazil' are atributes of the FeatureCollection table schema, 'country_na' stands for Country Name, and 'Brazil' stands for the name of the country that we want to get the boundaries, so for filtering we have to first select the way we want to access it, in this case by country name and then we have to select the country that we want. There are some other options in the way to access this data.

Based on the LSIB's table schema, we have the following options to use as filter:

<ul>
    <li>Country Code
        <ul>
            <li>Using the string 'country_co' on the filter</li>
            <li>You can access countries boundaries using their official code</li>
            <li>ee.Filter.eq('country_co', 'BR')</li>
        </ul>
    </li>
    <li>Country Name
        <ul>
            <li>Using the string 'country_na' on the filter</li>
            <li>You can access countries boundaries using the name</li>
            <li>ee.Filter.eq('country_na', 'Brazil')</li>
        </ul>
    </li>
    <li>World Region
        <ul>
            <li>Using the string 'wld_rgn'</li>
            <li>You can access boundaries of different continents</li>
            <li>ee.Filter.eq('wld_rgn', 'South America')</li>
        </ul>
</ul>

Some regions are too big or have too much polygons and load their data in an interactive map is not a good idea, if you try it will raise an error.

In [5]:
# Filtering to Brazil's boundaries
bound_brazil = countries.filter(ee.Filter.eq('country_na', 'Brazil'))

Now that we have the boundaries that we want, we need to acquire the images that we want. There are two ways of doing it, the first is similar to the method we applied on the FeatureCollection which we use the ImageCollection.filter() function and then use a ee.Filter as argument to the function. In this case, as we want to filter by date, we are using a filter as ee.Filter.date(), and selecting the time range that we are interested, for example a timestamp of 1 year, from the first day of 2017 to its last day. As the function ee.Filter.bounds(bound_brazil) is not working in Python Library yet, for filtering by bounds we just have one option.

The second way is to use functions from ImageCollection class that can make this process easier. So, instead of creating a filter and using it as argument to ee.Filter, we just call the functions .filterDate() and filterBounds(), that is a faster way to do the same thing.

In [6]:
# Filtering landsat-8 iamgery by date and by location using the boundaries we selected
dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1').filter(ee.Filter.date('2017-01-01', '2017-12-31')).filterBounds(bound_brazil) 
# doesn't work in python yet .filter(ee.Filter.bounds(bound_brazil))

In [7]:
dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1').filterDate('2017-01-01', '2017-12-31').filterBounds(bound_brazil)

All right, we got the images we want, let's see how many images we have.

In [8]:
dataset.size().getInfo()

6115

Yeah, that's a lot. But relax, we're not going to download or process all those images, we will transform all of them in only 1 image and then show that image on a map. For this, we first have to select the bands that we want, but what are those bands and which one we should select?

Sensors used on satellites can capture different light spectra, from visible spectrum to infrared. Each band or a group of bands can be used with different purpose, for example we can use the bands red, green and blue to get what is called the True Color image, that is the image as we see - the visible spectrum. Or we can use the Near Infrared and the Red bands to calculate the NDVI, as we saw in the previous notebook. So there are many use cases for each band and we will explore some of them in this path. However, in this notebook we will use the red, green and blue bands to create a True Color image. 

Each satellite has a different number of bands and they are organized differently, hence before starting working with any satellite we have to take a look on its bands. That said, let's take a look at Landsat-8's bands. It has 11 bands and one mask for quality assesment (QA) which provides some useful information about the state of each pixel. The bands are:

<ul>
    <li>Band 1 : Coastal Aerosol</li>
    <li>Band 2 : Blue</li>
    <li>Band 3 : Green</li>
    <li>Band 4 : Red</li>
    <li>Band 5 : Near infrared</li>
    <li>Band 6 : Shortwave infrared 1</li>
    <li>Band 7 : Shortwave infrared 2</li>
    <li>Band 8 : Band 8 Panchromatic</li>
    <li>Band 9 : Cirrus</li>
    <li>Band 10: Thermal Infrared 1</li>
    <li>Band 11: Thermal Infrared 2</li>
    <li>BQA    : Quality Assesment bitmask</li>
</ul>


<img src='Assets/true_false_color.png' width='600' height='500' style='margin-left: 500px;'>



Ok, ok, we already know what are bands and which one we have to select, but how to select them? An ImageCollection object has and select method that we can use to choose the bands we want, just using as argument a list of strings, each string being the band we want, for example Band 1: 'B1', Band 2: 'B2', Band Quality Assessment: 'BQA', etc.

In [9]:
# Selecting the bands we are going to use
true_color = dataset.select(['B4', 'B3', 'B2'])

For visualizing the images we selected, we have to define some visualization parameters creating a dictionary and setting some values to it. With these parameters we can manipulate the visualization of the images and features, change their color, the range of the pixels we want, etc. For images, we must define the min and max parameters, the min is the minimum value of the image which will be mapped to 0 and max is the maximum value of the image which will be mapped to 255. Remember that we selected the Raw Images and there are two more options, surface reflectance and top of atmosphere? The minimum and maximum values of image's pixels vary between those types of images, and also vary between different satellites. In this case, for Landsat-8 raw imagery, the minimum value is 0 and the maximum is 3000.

Other parameters are:

<ul>
    <li>bands : Comma-delimited list of three band names to be mapped to RGB</li>
    <li>min : Value(s) to map to 0</li>
    <li>max : Value(s) to map to 255</li>
    <li>gain : Value(s) by which to multiply each pixel value</li>
    <li>bias : Value(s) to add to each DN</li>
    <li>gamma : Gamma correction factor(s)</li>
    <li>palette : List of CSS-style color strings (single-band images only)</li>
    <li>opacity : The opacity of the layer (0.0 is fully transparent and 1.0 is fully opaque)</li>
</ul>

In the above cell we are defining three differente visualization parameters. The firts one is for the image, so we just need to set the minimum and maximum pixel's values. The second, for the feature collection, and in this case we are going to see a cool difference that will be explained above, and we set the color red to this feature. And the last one, we set also a color, but using palette, to the boundaries. You can change the colors the way you want and play with them and the other paramenters. This is a good way to understand better the way each parameter works.

In [10]:
# Defining the visualization parameters to the interactive map
true_color_vis = {
    'min' : 0.0,
    'max' : 30000.0,
}

feature_collection_vis = {
    'color' : 'red'
}

boundaries_vis = {
    'palette' : '0000FF'
}

Let's remember what we have until now: the images, that we filtered by date and by location; the feature collection with the boundaries from the location we want, which is Brazil; the bands we need to use to generate a true color image; the parameters to define as we want to see each one of those things; the interactive map. All right, I think that's all, we just need to put everything together now. For this, we are going to add layers for our interactive map, each layer containing the data we want to visualize.

In the first layer we are adding the images we selected. Actually we are using an ImageCollection as argument, but it is turned into a composite, using the median value for each pixel, of the whole ImageCollection. Simply as putting the object, the parameters and the name of the layer we want, we have this data added to our map.

Second, we are adding the whole filtered FeatureCollection. As you will see in red executing the following cells, the FeatureCollection is not only the boundaries, it takes the whole territory of the region we selected, and adding the FeatureCollection in this way you can visualize it.

Third, we are adding only the boundaries. For this, we have to use the function ee.Image().paint, to paint only the boundaries using the color we set.

In [11]:
# Adding the landsat-8' images layer
_map.addLayer(true_color, true_color_vis, 'True Color')
# Adding a layer to visualize the FeatureCollection
_map.addLayer(bound_brazil, feature_collection_vis, 'FeatureCollection')
# Adding a layer to visualize only the bondaries
_map.addLayer(ee.Image().paint(bound_brazil, color=0, width=3), boundaries_vis, 'Brazil')

This is a simple line, but a really cool one.

With this line we are able to use the map in fullscreen using the third button on the top left side of the map.

<img src='Assets/zoom.png' width='50' height='100' style='margin-left: 250px; border: 1px solid;'>

See the latitude and longitude of any point in the map just by clicking on it.

<img src='Assets/latlog.png' width='250' height='100' style='margin-left: 150px; border: 1px solid;'>

And, the coolest feature, we can control which layer we want to show using a button on the top right side of the map. You can play around with those features.

<img src='Assets/layers.png' width='150' height='100' style='margin-left: 200px; border: 1px solid;'>

In [12]:
# Setting folium's map control with geehydro
_map.setControlVisibility()

And finally the map!!

You can just run the cell and it will show our interactive map and you're free to play with it.

In [13]:
# Show the map
_map